# Naive_Bayes.ipynb

本 Notebook 將詳細介紹樸素貝葉斯 (Naive Bayes) 分類器，包括：

1. 貝葉斯定理基礎與 Naive Bayes 概念
2. Zero-Frequency 問題與平滑技術（如拉普拉斯平滑）
3. 高斯樸素貝葉斯 (GaussianNB)
4. 多項式樸素貝葉斯 (MultinomialNB)
5. 伯努利樸素貝葉斯 (BernoulliNB)
6. 互補樸素貝葉斯 (ComplementNB)
7. 實際應用案例（Iris 資料集、20 Newsgroups 文本分類、二元特徵合成資料）
8. Pipeline、Cross-Validation、模型選擇與實務建議

樸素貝葉斯分類器基於貝葉斯定理與特徵條件獨立假設，雖然假設強烈但實務中仍能取得不錯成效，特別在文本分類中表現良好，且計算速度快、對小資料集友好。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid", font_scale=1.2)

from sklearn.datasets import load_iris, fetch_20newsgroups, make_classification
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB, ComplementNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

## 1. 貝葉斯定理基礎

**貝葉斯定理**：

$$ P(y|x) = \frac{P(x|y)P(y)}{P(x)} $$

其中：
- $P(y|x)$：後驗機率 (posterior)，在給定 x 下 y 的機率。
- $P(x|y)$：似然 (likelihood)，在 y 下觀察到 x 的機率。
- $P(y)$：先驗機率 (prior)，不考慮 x 時，y 的機率。
- $P(x)$：證據 (evidence)，觀察到 x 的整體機率。

**樸素貝葉斯假設**：特徵間條件獨立。

$ P(x_1, x_2, ..., x_n | y) = \prod_{i=1}^{n} P(x_i|y) $

最終決策：

$ \hat{y} = \arg\max_y P(y) \prod_{i=1}^{n}P(x_i|y) $

**優點**：
- 訓練與預測速度快。
- 對小樣本表現佳。
- 對高維資料仍可行。

**缺點**：
- 特徵獨立假設在現實常不成立。
- Zero-Frequency 問題：若某類別特徵組合未出現在訓練集中，機率估計會為0。

### 1.1 簡單示例 (bayes_example)
透過一個簡單的醫學測試案例來示範貝葉斯定理計算後驗機率。

In [ ]:
def bayes_example():
    # 假設我們有以下數據：
    # P(cancer) = 0.01  患癌症的機率
    # P(positive|cancer) = 0.9 患癌症且測試陽性的機率
    # P(positive|no cancer) = 0.2 未患癌症但測試陽性的機率

    P_cancer = 0.01
    P_positive_given_cancer = 0.9
    P_positive_given_no_cancer = 0.2

    # P(positive) = P(positive|cancer)*P(cancer) + P(positive|no_cancer)*P(no_cancer)
    P_positive = P_positive_given_cancer * P_cancer + P_positive_given_no_cancer * (1 - P_cancer)

    # P(cancer|positive) = [P(positive|cancer)*P(cancer)] / P(positive)
    P_cancer_given_positive = (P_positive_given_cancer * P_cancer) / P_positive

    print("檢測陽性時患癌症的機率：{:.2%}".format(P_cancer_given_positive))

bayes_example()

### 1.2 Zero-Frequency 問題與平滑

Zero-Frequency 問題：如果訓練集中某特徵在某類別下從未出現，則該特徵與類別組合的條件機率估計為0，會導致整體後驗機率為0。

**解決方法**：拉普拉斯平滑 (Laplace smoothing) 或加法平滑 (Additive smoothing)，在計數中加上常數，以避免零計數。MultinomialNB、BernoulliNB中可透過 `alpha` 參數進行平滑。

## 2. GaussianNB 範例 (Iris 資料集)


In [ ]:
# 使用 Iris 資料集示範 GaussianNB
iris = load_iris()
X_i = iris.data
y_i = iris.target
X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(X_i, y_i, test_size=0.2, random_state=42)

gnb = GaussianNB()
gnb.fit(X_train_i, y_train_i)
y_pred_i = gnb.predict(X_test_i)
acc_i = accuracy_score(y_test_i, y_pred_i)
print("GaussianNB (Iris):")
print(f"Accuracy: {acc_i:.4f}")
print(classification_report(y_test_i, y_pred_i, target_names=iris.target_names))

# 混淆矩陣
cm_i = confusion_matrix(y_test_i, y_pred_i)
sns.heatmap(cm_i, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (GaussianNB)')
plt.show()

## 3. MultinomialNB 範例 (文本分類)

使用 20 Newsgroups 部分類別示範多項式樸素貝葉斯，用 CountVectorizer 將文本轉換為詞頻特徵。

In [ ]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)

vec = CountVectorizer(stop_words='english')
X_train_t = vec.fit_transform(newsgroups_train.data)
X_test_t = vec.transform(newsgroups_test.data)
y_train_t = newsgroups_train.target
y_test_t = newsgroups_test.target

mnb = MultinomialNB()
mnb.fit(X_train_t, y_train_t)
y_pred_t = mnb.predict(X_test_t)
acc_t = accuracy_score(y_test_t, y_pred_t)
print("MultinomialNB (20 Newsgroups):")
print(f"Accuracy: {acc_t:.4f}")
print(classification_report(y_test_t, y_pred_t, target_names=newsgroups_train.target_names))

cm_t = confusion_matrix(y_test_t, y_pred_t)
sns.heatmap(cm_t, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (MultinomialNB)')
plt.show()

## 4. BernoulliNB 範例

BernoulliNB 適用二元特徵（詞有無），在文本分類中可使用 `binary=True` 參數產生二元特徵。

In [ ]:
bnb = BernoulliNB()
vec_bin = CountVectorizer(binary=True, stop_words='english')
X_train_bi = vec_bin.fit_transform(newsgroups_train.data)
X_test_bi = vec_bin.transform(newsgroups_test.data)
bnb.fit(X_train_bi, y_train_t)
y_pred_bi = bnb.predict(X_test_bi)
acc_bi = accuracy_score(y_test_t, y_pred_bi)
print("BernoulliNB (20 Newsgroups):")
print(f"Accuracy: {acc_bi:.4f}")
print(classification_report(y_test_t, y_pred_bi, target_names=newsgroups_train.target_names))

cm_bi = confusion_matrix(y_test_t, y_pred_bi)
sns.heatmap(cm_bi, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (BernoulliNB)')
plt.show()

## 5. ComplementNB 範例

ComplementNB 是 MultinomialNB 的變體，對不平衡數據更穩定。


In [ ]:
cnb = ComplementNB()
cnb.fit(X_train_t, y_train_t)
y_pred_c = cnb.predict(X_test_t)
acc_c = accuracy_score(y_test_t, y_pred_c)
print("ComplementNB (20 Newsgroups):")
print(f"Accuracy: {acc_c:.4f}")
print(classification_report(y_test_t, y_pred_c, target_names=newsgroups_train.target_names))

cm_c = confusion_matrix(y_test_t, y_pred_c)
sns.heatmap(cm_c, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (ComplementNB)')
plt.show()

## 6. 使用合成資料 (Binary Classification) + GridSearchCV

示範對 MultinomialNB 使用 GridSearchCV 尋找最佳 alpha (平滑參數)。

In [ ]:
X_bin, y_bin = make_classification(n_samples=500, n_features=5, random_state=42)
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X_bin, y_bin, test_size=0.2, random_state=42)

# MultinomialNB要求非負值特徵，將特徵轉為正值
X_train_b_pos = np.abs(X_train_b) + 1
X_test_b_pos = np.abs(X_test_b) + 1

param_grid = {'alpha': [0.1, 0.5, 1.0, 2.0]}
mnb_gs = GridSearchCV(MultinomialNB(), param_grid, cv=5, scoring='accuracy')
mnb_gs.fit(X_train_b_pos, y_train_b)
print("最佳參數:", mnb_gs.best_params_)
y_pred_gs = mnb_gs.predict(X_test_b_pos)
print("Test Accuracy:", accuracy_score(y_test_b, y_pred_gs))

## 7. Pipeline、Cross-Validation 示例

以 GaussianNB 為例，進行 Cross-Validation 評估：

In [ ]:
X_iris_all = iris.data
y_iris_all = iris.target

pipe_gnb = Pipeline([
    ('scaler', StandardScaler()),
    ('gnb', GaussianNB())
])

scores = cross_val_score(pipe_gnb, X_iris_all, y_iris_all, cv=5, scoring='accuracy')
print("Cross-Validation Accuracy (GaussianNB on Iris):", scores)
print("Mean Accuracy:", np.mean(scores))

## 8. 實務建議與總結

### 8.1 模型選擇
- GaussianNB：連續特徵，特徵分佈近似高斯
- MultinomialNB：文本分類（詞頻特徵），非負離散特徵
- BernoulliNB：二元特徵，是否出現特徵
- ComplementNB：文本分類，不平衡資料更穩定

### 8.2 Zero-Frequency問題
使用 `alpha` 進行平滑 (拉普拉斯平滑) 避免零機率問題。

### 8.3 優點與缺點
優點：
- 計算快速、訓練與預測效率高
- 對高維、少量資料良好
- 易於理解與實現

缺點：
- 特徵獨立假設不常成立
- 對複雜關係可能無法充分捕捉

### 8.4 使用建議
- 文本分類常首選 MultinomialNB 或 BernoulliNB
- 對連續特徵建議 GaussianNB
- 不平衡文本資料可考慮 ComplementNB
- 使用交叉驗證、GridSearchCV 調整參數（如 alpha）
- 搭配特徵工程(詞頻、TF-IDF、標準化)提昇效能

### 參考資料
- [Scikit-learn Documentation: Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html)
- [Scikit-learn Documentation: 20 Newsgroups text dataset](https://scikit-learn.org/stable/datasets/real_world.html#the-20-newsgroups-text-dataset)